# Introduction

#### Lily is travelling to Manhattan the next month, and  she will spend about 2 weeks there. She wants to enjoy her stay and find a great place to live during the trip
#### She'd like to find a neighborhood with many restaurants and also a good hotel to live in

-----------------------------------------------------
# Data

#### We will use the Manhattan neighborhood data from the lab( https://geo.nyu.edu/catalog/nyu_2451_34572) which contains the Borough, the name and latittude, longitude of each neighborhood in Manhattan
#### and we will also utilize the Foursquare API, to group these neighborhoods into clusters (for example: 5 clusters), so that we can have different clusters of neighborhoods that are similar within the cluster yet different from other clusters
#### Review the clusters and find one neighborhood form the most suitable cluster that has many restaurants and recommend a good hotel for Lily.


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Libraries imported.


##### Tranform the data into a pandas dataframe

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
neighborhoods_data = newyork_data['features']

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
neighborhoods.head()

Data downloaded!


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [3]:
neighborhoods.shape

(306, 4)

##### Neighborhoods in Manhattan

In [4]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688




Use geopy library to get the latitude and longitude values of Manhattan

In [5]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


In [6]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

#### now we have the neighborhood data(name,Latitude,Longitude of the neighborhood in Manhattan), next we will utilize the foursquare API to further explore/search the area

##### Explore Neighborhoods in Manhattan

In [7]:
# @hide cell
CLIENT_ID = 'YCZ2PFBDENOXZUX25S41HS3ZWHGMA4WDY3BTDEXFMDUFZJUB' # your Foursquare ID
CLIENT_SECRET = 'U1EUP1HXLFDAZXUZVTNPTXDGTSBZD3BPBLGFOY5WWPLBJSNF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


In [8]:
radius=500
LIMIT=100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


## Now we already have the venue information for each neighborhood in Manhattan, we can explore and research more in the following steps!~

In [10]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3328, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Dunkin' Donuts,40.877136,-73.906666,Donut Shop
4,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop


In [11]:
#manhattan_venues['Venue Category'].unique()
len(manhattan_venues['Venue Category'].unique())


327

In [12]:
#manhattan_venues[manhattan_venues['Venue Category'].str.contains("Restaurant")]

In [13]:
#check how many restaurants each neighborhood has
restaurant_count=manhattan_venues[manhattan_venues['Venue Category'].str.contains("Restaurant")].groupby('Neighborhood').count()[['Venue']]
restaurant_count.rename(columns={"Venue": "restaruant_count"},inplace=True)
restaurant_count.reset_index(inplace=True)
restaurant_count

,Neighborhood,restaruant_count
0,Battery Park City,9
1,Carnegie Hill,22
2,Central Harlem,14
3,Chelsea,26
4,Chinatown,42
5,Civic Center,22
6,Clinton,23
7,East Harlem,16
8,East Village,39
9,Financial District,22


In [14]:
#check how many hotel each neighborhood has
Hotel_count=manhattan_venues[manhattan_venues['Venue Category']=="Hotel"].groupby('Neighborhood').count()[['Venue']]
Hotel_count.rename(columns={"Venue": "hotel_count"},inplace=True)
Hotel_count.reset_index(inplace=True)
Hotel_count

,Neighborhood,hotel_count
0,Battery Park City,5
1,Chelsea,3
2,Chinatown,1
3,Civic Center,2
4,Clinton,4
5,Financial District,5
6,Gramercy,2
7,Greenwich Village,1
8,Hudson Yards,4
9,Lincoln Square,2


The neighborhood must have at least one hotel

In [15]:
manhattan_venues=pd.merge(manhattan_venues, Hotel_count[['Neighborhood']], how='inner', on='Neighborhood')
manhattan_venues.shape

(2056, 7)

In [16]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Business Service,Butcher,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Theater,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Resort,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soba Restaurant,Social Club,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Tourist Information Center,Toy / Game Store,Trail,Tree,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Volleyball Court,Watch Shop,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [17]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Business Service,Butcher,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Theater,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Resort,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soba Restaurant,Social Club,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Tourist Information Center,Toy / Game Store,Trail,Tree,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Volleyball Court,Watch Shop,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Battery Park City,0.00,0.00,0.010101,0.00,0.00,

In [18]:
manhattan_grouped.shape

(21, 268)

find the top 10 most common venues for each neighborhood

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Coffee Shop,Hotel,Gym,Wine Shop,Italian Restaurant,Shopping Mall,Playground,BBQ Joint,Memorial Site
1,Chelsea,Coffee Shop,Italian Restaurant,Ice Cream Shop,Nightclub,Bakery,American Restaurant,Theater,Hotel,Seafood Restaurant,Cupcake Shop
2,Chinatown,Chinese Restaurant,Dim Sum Restaurant,American Restaurant,Vietnamese Restaurant,Cocktail Bar,Bubble Tea Shop,Hotpot Restaurant,Ice Cream Shop,Salon / Barbershop,Bakery
3,Civic Center,Gym / Fitness Center,Italian Restaurant,Bakery,French Restaurant,Yoga Studio,Sandwich Place,Sporting Goods Shop,American Restaurant,Spa,Park
4,Clinton,Theater,Italian Restaurant,American Restaurant,Hotel,Gym / Fitness Center,Wine Shop,Spa,Gym,Indie Theater,Lounge


#### Cluster neighborhoods

In [30]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 2, 2, 2, 4, 3, 2, 2, 3, 4], dtype=int32)

In [31]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


In [32]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,3,Battery Park City,Park,Coffee Shop,Hotel,Gym,Wine Shop,Italian Restaurant,Shopping Mall,Playground,BBQ Joint,Memorial Site
1,2,Chelsea,Coffee Shop,Italian Restaurant,Ice Cream Shop,Nightclub,Bakery,American Restaurant,Theater,Hotel,Seafood Restaurant,Cupcake Shop
2,2,Chinatown,Chinese Restaurant,Dim Sum Restaurant,American Restaurant,Vietnamese Restaurant,Cocktail Bar,Bubble Tea Shop,Hotpot Restaurant,Ice Cream Shop,Salon / Barbershop,Bakery
3,2,Civic Center,Gym / Fitness Center,Italian Restaurant,Bakery,French Restaurant,Yoga Studio,Sandwich Place,Sporting Goods Shop,American Restaurant,Spa,Park
4,4,Clinton,Theater,Italian Restaurant,American Restaurant,Hotel,Gym / Fitness Center,Wine Shop,Spa,Gym,Indie Theater,Lounge


In [34]:
#manhattan_merged=manhattan_data
#manhattan_merged = pd.merge(manhattan_merged,manhattan_data[['Borough','Neighborhood','Latitude','Longitude']],on='Neighborhood')

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
#manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
manhattan_merged = pd.merge(manhattan_data[['Borough','Neighborhood','Latitude','Longitude']],neighborhoods_venues_sorted,on='Neighborhood')

manhattan_merged.head() # check the last columns!
#manhattan_merged.shape

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Chinatown,40.715618,-73.994279,2,Chinese Restaurant,Dim Sum Restaurant,American Restaurant,Vietnamese Restaurant,Cocktail Bar,Bubble Tea Shop,Hotpot Restaurant,Ice Cream Shop,Salon / Barbershop,Bakery
1,Manhattan,Upper East Side,40.775639,-73.960508,2,Italian Restaurant,Exhibit,Coffee Shop,Bakery,Juice Bar,Gym / Fitness Center,Cocktail Bar,French Restaurant,Art Gallery,Hotel
2,Manhattan,Lincoln Square,40.773529,-73.985338,4,Theater,Concert Hall,Italian Restaurant,Plaza,Gym / Fitness Center,Café,French Restaurant,Opera House,Indie Movie Theater,Performing Arts Venue
3,Manhattan,Clinton,40.759101,-73.996119,4,Theater,Italian Restaurant,American Restaurant,Hotel,Gym / Fitness Center,Wine Shop,Spa,Gym,Indie Theater,Lounge
4,Manhattan,Midtown,40.754691,-73.981669,1,Hotel,Clothing Store,Cocktail Bar,Theater,Steakhouse,American Restaurant,Food Truck,Bakery,Bookstore,Spa


In [35]:
manhattan_merged.shape

(21, 15)

In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [36]:
manhattan_merged.groupby('Cluster Labels').count()

,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Cluster Labels,,,,,,,,,,,,,,
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,2,2,2,2,2,2,2,2,2,2,2,2,2,2
2,9,9,9,9,9,9,9,9,9,9,9,9,9,9
3,7,7,7,7,7,7,7,7,7,7,7,7,7,7
4,2,2,2,2,2,2,2,2,2,2,2,2,2,2


In [46]:
#manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

In [44]:
#manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1]

In [47]:
#manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2]

In [48]:
#manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3]

In [45]:
#manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4]

In [79]:
manhattan_merged['combined'] =manhattan_merged[manhattan_merged.columns[5:14]].apply(lambda x: ','.join(x), axis=1)
manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,combined
0,Manhattan,Chinatown,40.715618,-73.994279,2,Chinese Restaurant,Dim Sum Restaurant,American Restaurant,Vietnamese Restaurant,Cocktail Bar,Bubble Tea Shop,Hotpot Restaurant,Ice Cream Shop,Salon / Barbershop,Bakery,"Chinese Restaurant,Dim Sum Restaurant,American..."
1,Manhattan,Upper East Side,40.775639,-73.960508,2,Italian Restaurant,Exhibit,Coffee Shop,Bakery,Juice Bar,Gym / Fitness Center,Cocktail Bar,French Restaurant,Art Gallery,Hotel,"Italian Restaurant,Exhibit,Coffee Shop,Bakery,..."
2,Manhattan,Lincoln Square,40.773529,-73.985338,4,Theater,Concert Hall,Italian Restaurant,Plaza,Gym / Fitness Center,Café,French Restaurant,Opera House,Indie Movie Theater,Performing Arts Venue,"Theater,Concert Hall,Italian Restaurant,Plaza,..."
3,Manhattan,Clinton,40.759101,-73.996119,4,Theater,Italian Restaurant,American Restaurant,Hotel,Gym / Fitness Center,Wine Shop,Spa,Gym,Indie Theater,Lounge,"Theater,Italian Restaurant,American Restaurant..."
4,Manhattan,Midtown,40.754691,-73.981669,1,Hotel,Clothing Store,Cocktail Bar,Theater,Steakhouse,American Restaurant,Food Truck,Bakery,Bookstore,Spa,"Hotel,Clothing Store,Cocktail Bar,Theater,Stea..."


In [80]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,combined
16,Manhattan,Midtown South,40.74851,-73.988713,0,Korean Restaurant,Hotel,Hotel Bar,Japanese Restaurant,Coffee Shop,Cosmetics Shop,Bakery,Cocktail Bar,Boutique,Gym / Fitness Center,"Korean Restaurant,Hotel,Hotel Bar,Japanese Res..."


In [81]:
search_query = 'Hotel'
radius = 500
print(search_query + ' .... OK!')

Hotel .... OK!


In [87]:
latitude=manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0]['Latitude'].get_value
longitude=manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0]['Longitude'].get_value
print(latitude)

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

<bound method Series.get_value of 16    40.74851
Name: Latitude, dtype: float64>


'https://api.foursquare.com/v2/venues/search?client_id=YCZ2PFBDENOXZUX25S41HS3ZWHGMA4WDY3BTDEXFMDUFZJUB&client_secret=U1EUP1HXLFDAZXUZVTNPTXDGTSBZD3BPBLGFOY5WWPLBJSNF&ll=<bound method Series.get_value of 16    40.74851\nName: Latitude, dtype: float64>,<bound method Series.get_value of 16   -73.988713\nName: Longitude, dtype: float64>&v=20180605&query=Hotel&radius=500&limit=100'

In [88]:
results = requests.get(url).json()
venues = results['response']['venues']

# tranform venues into a dataframe
df = json_normalize(venues)
df.head()

KeyError: 'venues'